<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TestET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemple d'apprentissage du ET avec TensorFlow
## Illustration de l'importance du Bias

|a|b|a et b|
|:-:|:-:|:-:|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|


*Théoriquement, en 1 couche, l'apprentissage du ET par réseau de neurones n'est pas possible.*

En effet, la couche n'est consituée que de 1 neurone (1 sortie), ses entrées sont les valeurs `a` et `b`.
`wa` et `wb` étant les poids affectés à ces valeurs, il faut vérifier : 
 - `f(0)` tend vers 0 --> ok
 - `f(wb)` tend vers 0
 - `f(wa)` tend vers 0
 - `f(wa + wb)` tend vers 1 --> conflit avec les lignes précédentes
 
*Vérifions le...*


---
**Importer les librairies**

In [1]:
#keras : Python Deep Learning library
import tensorflow.keras as keras
#prevision d'utiliser un réseau en couches séquentielles
from tensorflow.keras.models import Sequential
#prevision d'utiliser des couches totalement connectées la précédente
from tensorflow.keras.layers import Dense
#utilisation de la classique librairie pour tableaux, ...
import numpy as np

---

## Définir les entrées et sorties attendues


In [2]:
# a et b sont les seules entrées
entrees = np.array([[0,0],[0,1],[1,0],[1,1]], float)

# une seule sortie
sorties = np.array([[0],[0],[0],[1]], float)

---
## 1. Version sans BIAS

### 1.1. Choisir le modèle de réseau 
***ici les couches sont séquentielles***

In [3]:
model = Sequential()

### 1.2. Définir l'architecture du réseau
- ici une seule couche constituée de 1 neurone en sortie, 
- de 2 neurones en entrée (pour chaque valeur), 
- utilisation de la sigmoïde comme fonction d'activation

In [22]:
model.add(Dense(1, input_dim=2, use_bias=False, activation='sigmoid'))

c:\Program Files\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


---

### 1.3. Compiler le  réseau
Ici, on précise que 
  - l'algo de correction d'erreur est 'Adamax', 
  - l'erreur calculée est la moyenne des erreurs commises au carrées. $E = \Sigma_{i=1 \dots n}{(y^I_i - y_i)} / n$, $y^I_i$ sortie idéale attendue, $y_i$ sortie calculée

In [23]:
model.compile(optimizer='adamax', loss='mse')

---

### 1.4. Entraîner le réseau 
- ici on n'affiche pas les étapes de l'apprentissage, 
- et on lance 6000 cycles d'apprentissage (attendre entre 4 à 6mn !)

In [24]:
model.fit(entrees, sorties, verbose=0, epochs=6000)

---

### 1.5. Vérifier le réseau
Etape facultative, en général ***on teste le réseau sur d'autres exemples***. 
- Ici, on n'en a pas. Alors on lui demande de calculer la sortie pour chaque exemple de l'ensemble d'entraînement

In [25]:
predictions = model.predict(entrees)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [26]:
entrees

array([[0., 0.],
       [0., 1.],
       [1., 0.],
       [1., 1.]])

In [27]:
predictions

array([[0.5000722 ],
       [0.51060617],
       [0.51060617],
       [0.92101085]], dtype=float32)

---
### 1.6. Affichage des résultats
Ici pas de nécessité de graphique d'évolution de l'erreur.
On affiche les entrées, la sortie attendue, la sortie calculée ainsi que les poids appliquées aux entrées et au signal bias..

In [28]:
def verification(bias=False):
    print("verification")
    for i in range(0, len(entrees)):
        print(entrees[i][0], " - ", entrees[i][1], " attendu ", sorties[i], " trouvé ",  predictions[i])

    ws = model.get_weights()
    print("poids pour entree x = " , ws[0][0][0])
    print("poids pour entree y = " , ws[0][1][0])
    if(bias):print("poids pour bias = ", ws[1][0])

verification()

verification
0.0  -  0.0  attendu  [0.]  trouvé  [0.5000722]
0.0  -  1.0  attendu  [0.]  trouvé  [0.51060617]
1.0  -  0.0  attendu  [0.]  trouvé  [0.51060617]
1.0  -  1.0  attendu  [1.]  trouvé  [0.92101085]
poids pour entree x =  5.0005126
poids pour entree y =  5.0005136


In [29]:
loss = model.evaluate(entrees, sorties,verbose=0)
print("perte=",loss)

perte= 0.1944372057914734


**Des erreurs importantes donc**, comme prévu....

---
## 2. Version AVEC BIAS

Le tableau est alors

|bias|a|b|a et b|
|:-:|:-:|:-:|:-:|
|1|0|0|0|
|1|0|1|0|
|1|1|0|0|
|1|1|1|1|


*Théoriquement, en 1 couche, l'apprentissage du ET par réseau de neurones est alors possible.*

En effet, la couche n'est constituée que de 1 neurone (1 sortie), ses entrées sont les valeurs `bias`, `a` et `b`.
`wbias`, `wa` et `wb` étant les poids affectés à ces valeurs, il faut vérifier : 
 - `f(bias)` tend vers 0
 - `f(bias + wb)` tend vers 0
 - `f(bias + wa)` tend vers 0
 - `f(bias + wa + wb)` tend vers 1 
 
*Vérifions le...*



---

### 2.1. Définir l'architecture du réseau
- ici une seule couche constituée de 1 neurone en sortie, 
- de 3 neurones en entrée (2 contenant les valeurs + **un Bias** (émettant toujours le signal 1)), 
- utilisation de la sigmoide comme fonction d'activation

In [12]:
model = Sequential()
model.add(Dense(1, input_dim=2, use_bias=True, activation='sigmoid'))

---

### 2.2 Compiler et entraîner le  réseau 
Ici, on précise que 
  - l'algo de correction d'erreur est 'Adamax', 
  - l'erreur calculée utilise est la moyenne des erreurs au carrées 
  - on lance 6000 cycles d'apprentissage (attendre entre 4 à 6mn !)

In [13]:
model.compile(optimizer='adamax', loss='mse')

In [14]:
test = model.fit(entrees, sorties, verbose=0, epochs=6000)

---

### 2.2. Vérifier le réseau
Pas d'exemples de validation, on vérifie simplement la correspondance entre sortie attendue et la sortie réelle.

In [15]:
predictions = model.predict(entrees)
verification(True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
verification
0.0  -  0.0  attendu  [0.]  trouvé  [0.00241795]
0.0  -  1.0  attendu  [0.]  trouvé  [0.11837828]
1.0  -  0.0  attendu  [0.]  trouvé  [0.11836719]
1.0  -  1.0  attendu  [1.]  trouvé  [0.8814833]
poids pour entree x =  4.0144305
poids pour entree y =  4.014537
poids pour bias =  -6.0224147


In [16]:
loss = model.evaluate(entrees, sorties, verbose=0)
print("perte=",loss)

perte= 0.010519064962863922


**Apprentissage quasi parfait !!!** 
- -> démonstration concrère de l'effet du `Bias` !!

---
### Utilisation
Testons d'autres valeurs

In [17]:
tests = np.array([[0.5,0.2], [0.9, 0.7], [0.1, 0.9]], float)

In [18]:
predictions = model.predict(tests)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [19]:
predictions

array([[0.03870576],
       [0.5988674 ],
       [0.11837719]], dtype=float32)

In [20]:
def print_predictions(entrees, sorties):
    for i in range(0, len(entrees)):
        print(f"apparition d'une données vraie à {entrees[i][0]*100:.2f}% et une vraie à {entrees[i][1]*100:.2f}% en même temps est possible à {sorties[i][0]*100:.2f}%")

In [21]:
print_predictions(tests, predictions)

apparition d'une données vraie à 50.00% et une vraie à 20.00% en même temps est possible à 3.87%
apparition d'une données vraie à 90.00% et une vraie à 70.00% en même temps est possible à 59.89%
apparition d'une données vraie à 10.00% et une vraie à 90.00% en même temps est possible à 11.84%
